**加入原配点的邻点，扩充为新的配点**

In [26]:
%reset -f

In [27]:
import sys, os
sys.path.append(os.path.dirname(os.path.realpath('../utils')))
import numpy as np
from utils.aiwt import cjkfun

In [28]:
J = 15
j0 = 0

a = 0
b = 2
Re = 1000

deltax = (b-a)/(2**J)
XJ0 = np.arange(a, b+deltax, deltax).reshape(1,-1)
V0 = np.sin(np.pi * XJ0)

M = 1
CC = 1

t = 0.4
tao = 0.001

eps0 = 0.005
window = 3

same = np.inf

nn_dict = {1:[1,'0.01'], 400:[3,'0.4'], 600:[5,'0.6'], 800:[1,'0.8'],
           1200:[3,'1.2'], 600:[5,'1.6'], 2000:[1,'2.0'], 2400:[3,'2.4'],
           2600:[5,'2.6'], 2800:[1,'2.8'], 3000:[3,'3.0'], 3200:[5,'3.2']}
cjk, cjknum, zJc, zJcnum = cjkfun(V0, J, j0, a, b, eps0)

In [ ]:
# gjk, gjknum, gJc, gJcnum = z_g(j0, J, cjk, cjknum, zJc, zJcnum, a, b, M, CC)

In [ ]:
def z_g(j0, J, zjk, zjknum, zJc, zJcnum, a, b, M, CC):
    """
    输入：
        j0 : int -- 初始层数
        J : int -- 小波插值层数
        zjk : list[ndarray(-1), ...] -- j层每层的所有\alpha不为0的配点位置索引
        zjknum : list -- 层配点索引数量（每层需计算的配点数量）
        zJc : list -- \alpha不为0的配点实际位置
        zJcnum : int -- 实际位置的数量
        a, b : int -- 函数的限制范围
        M : int -- 上下为M个间隔的邻层
        CC : int -- 左右第CC个间隔的邻点
    功能：
        找 原jm层 所有配点的左右邻点
        邻点的寻找策略为与配点间隔为某固定值的点
        固定值为: 该层以上高M级别内所有层配点的（间隔长度/2） *CC
        找到后放入jm的高一层
    输出：
        gjk : list[ndarray(-1), ...] -- 加入邻点后的j层每层所有配点位置索引
        gjknum : list -- 加入邻点后的层配点索引数量（每层需计算的配点数量）
        gJc : list -- 加入邻点后的配点实际位置
        gJcnum : int -- 加入邻点后的配点实际位置的数量
    """
    gJc = zJc.copy()
    gJcnum = zJcnum
    gjk = zjk.copy()
    gjknum = zjknum.copy()
    for j in range(j0, J):
        xj = j+1
        deltax = (b - a) / (2 ** xj)
        jmin = max(j0+1, xj-M) 
        jmax = min(J, xj+M)
        for k in range(zjknum[j]):
            kk = zjk[j][k]
            xk = a + deltax * (2*kk + 1)
            jMax = len(gjk)
            xmin0 = xk - CC*(0.5**xj)
            xmax0 = xk + CC*(0.5**xj)
            for jj in range(jmin, jmax+1):
                deltaxjj=(b-a)/(2**jj)
                xmin = max(xmin0, a+deltaxjj)
                xmax = min(xmax0, b-deltaxjj)
                kmin = np.ceil((xmin-a)/deltaxjj)
                kmax = np.floor((xmax-a)/deltaxjj)
                if kmin < kmax:
                    delta = 2 ** (J - jj)
                    temp = np.arange(kmin*delta, kmax*delta+1, delta).tolist()
                    gJc = np.union1d(gJc, temp)
                    oe = np.mod(kmin,2)
                    if oe == 1:
                        jm = jj - 1
                        if jm >= j0:
                            temp1 = np.array([])
                            if jm <= jMax - 1:
                                temp1 = np.append(temp1, gjk[jm], 0)
                            temp2 = np.arange((kmin - 1) / 2, (kmax - 1) / 2 + 1)
                            temp3 = np.union1d(temp1,temp2)
                            temp3 = np.array(temp3, dtype=np.int64)
                            if jm < len(gjk):
                                gjk[jm] = temp3
                            else:
                                gjk.append(temp3) 
                            gjknum[jm] = temp3.shape[0]
                    elif oe == 0:
                        jm = jj - 2
                        if jm >= j0:
                            if jm > jMax - 1:
                                temp1 = np.array([])
                            else:
                                temp1 = np.append(np.array([]), gjk[jm], 0)
                            temp2 = np.arange((kmin - 2) / 4, (kmax - 2) / 4 + 1)
                            temp3 = np.union1d(temp1,temp2)
                            temp3 = np.array(temp3, dtype=np.int64)
                            if jm < len(gjk):
                                gjk[jm] = temp3
                            else:
                                gjk.append(temp3) 
                            gjknum[jm] = temp3.shape[0]
    gJcnum = gJc.shape[0]
    return gjk, gjknum, gJc, gJcnum

In [ ]:
gjk, gjknum, gJc, gJcnum = z_g(j0, J, cjk, cjknum, zJc, zJcnum, a, b, M, CC)